# IRIS-HEP CMS Integration Challenge: Workflow Demonstration

This notebook demonstrates the workflow of the IRIS-HEP CMS integration challenge. The workflow is modular and consists of the following steps:

1. Environment Setup
2. Configuration Construction
3. Metadata Extraction
4. Skimming
5. Histogramming
6. Statistical Analysis
7. Plotting

## 1. Environment Setup

To set up the environment, use the following command to launch JupyterLab with the required dependencies:

```sh
pixi run lab
```

Alternatively, if you prefer a Conda-like environment, activate it using the provided script:

```sh
source pixi_activate.sh
```

## 2. Configuration Construction

A lightweight Python config defines general settings, datasets, cuts, observables, and systematics — all are type-checked and (mostly) CLI-overrideable.
Everything else in the workflow reads from it.

In [1]:
# Import Rich-based Configuration Display from logging module
from utils.logging import display_config_table, get_config_logger

# Create a global config logger instance for this notebook
config_logger = get_config_logger()

In [2]:
# Example: Demonstrate configuration comparison and change detection
from user.configuration import config as original_config
from utils.schema import Config, load_config_with_restricted_cli

import copy

# Save the original configuration for comparison
config = copy.deepcopy(original_config)

print("=== Full configuration ===")
display_config_table(config, expand=True)

=== Full configuration ===


                                                 Configuration: GENERAL                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                          ┃ Value                                                                             
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ lumi                         │ 16400                                                                             
│ weight_branch                │ 'genWeight'                                                                       
│ analysis                     │ 'nondiff'                                                                         
│ run_skimming                 │ False                                                                             
│ run_histogramming            │ True                                                                              
│ run_statistics               │ True                                                                              
│ run_systematics              │ True                                                                              
│ run_plots_only               │ False                                                                             
│ run_mva_training             │ False                                                                             
│ run_metadata_generation      │ False                                                                             
│ read_from_cache              │ True                                                                              
│ output_dir                   │ 'example/outputs/'                                                                
│ lumifile                     │ './corrections/Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt'         
│ cache_dir                    │ '/tmp/integration/'                                                               
└──────────────────────────────┴───────────────────────────────────────────────────────────────────────────────────

                                               Configuration: PREPROCESS                                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                           ┃ Value                                                                            
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ branches                      │ {                                                                                
│                               │   Muon:                                                                          
│                               │ [                                                                                
│                               │     [0]: 'pt'                                                                    
│                               │     [1]: 'eta'                                                                   
│                               │     [2]: 'phi'                                                                   
│                               │     [3]: 'mass'                                                                  
│                               │     [4]: 'miniIsoId'                                                             
│                               │     [5]: 'tightId'                                                               
│                               │     [6]: 'charge'                                                                
│                               │   ]                                                                              
│                               │   FatJet: ['particleNet_TvsQCD', 'pt', 'eta', 'phi', 'mass']                     
│                               │   Jet:                                                                           
│                               │ [                                                                                
│                               │     [0]: 'btagDeepB'                                                             
│                               │     [1]: 'jetId'                                                                 
│                               │     [2]: 'pt'                                                                    
│                               │     [3]: 'eta'                                                                   
│                               │     [4]: 'phi'                                                                   
│                               │     [5]: 'mass'                                                                  
│                               │   ]                                                                              
│                               │   PuppiMET: ['pt', 'phi']                                                        
│                               │   HLT: ['TkMu50']                                                                
│                               │   Pileup: ['nTrueInt']                                                           
│                               │   event: ['genWeight', 'run', 'luminosityBlock', 'event']                        
│                               │ }                                                                                
│ ignore_missing                │ False                                                                            
│ mc_branches                   │ {                                                                                
│                               │   event: ['genWeight']                                                           
│                               │   Pileup: ['nTrueInt']                                                           
│                               │ }                     

                                           Configuration: BASELINE_SELECTION                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ function                          │ Function: Zprime_hardcuts_no_fj                                              
│ use                               │ [('Muon', None), ('Jet', None)]                                              
└───────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────

                                            Configuration: GOOD_OBJECT_MASKS                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ analysis                          │ [                                                                            
│                                   │   [0]:                                                                       
│                                   │ {                                                                            
│                                   │     object: 'Muon'                                                           
│                                   │     function: Function: <lambda>                                             
│                                   │     use: [('Muon', None)]                                                    
│                                   │   }                                                                          
│                                   │   [1]:                                                                       
│                                   │ {                                                                            
│                                   │     object: 'Jet'                                                            
│                                   │     function: Function: <lambda>                                             
│                                   │     use: [('Jet', None)]                                                     
│                                   │   }                                                                          
│                                   │   [2]:                                                                       
│                                   │ {                                                                            
│                                   │     object: 'FatJet'                                                         
│                                   │     function: Function: <lambda>                                             
│                                   │     use: [('FatJet', None)]                                                  
│                                   │   }                                                                          
│                                   │ ]                                                                            
│ mva                               │ [                                                                            
│                                   │   [0]:                                                                       
│                                   │ {                                                                            
│                                   │     object: 'Muon'                                                           
│                                   │     function: Function: <lambda>                                             
│                                   │     use: [('Muon', None)]                                                    
│                                   │   }                                                                          
│                                   │   [1]:                                                                       
│                                   │ {                                                                            
│                                   │     object: 'FatJet'                                                         
│                                   │     function: Func

...

                                               Configuration: CORRECTIONS                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ Items                             │ [                                                                            
│                                   │   [0]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'pu_weight'                                                        
│                                   │     file: 'corrections/puWeights.json.gz'                                    
│                                   │     type: 'event'                                                            
│                                   │     use: [('Pileup', 'nTrueInt')]                                            
│                                   │     op: 'mult'                                                               
│                                   │     key: 'Collisions16_UltraLegacy_goldenJSON'                               
│                                   │     use_correctionlib: True                                                  
│                                   │   }                                                                          
│                                   │   [1]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'muon_id_sf'                                                       
│                                   │     file: 'corrections/muon_Z.json.gz'                                       
│                                   │     use: [('Muon', 'eta'), ('Muon', 'pt')]                                   
│                                   │     transform: Function: <lambda>                                            
│                                   │     type: 'event'                                                            
│                                   │     key: 'NUM_TightID_DEN_TrackerMuons'                                      
│                                   │     use_correctionlib: True                                                  
│                                   │     op: 'mult'                                                               
│                                   │     up_and_down_idx: ['systup', 'systdown']                                  
│                                   │   }                                                                          
│                                   │ ]                                                                            
└───────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────

                                               Configuration: SYSTEMATICS                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ Items                             │ [                                                                            
│                                   │   [0]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'jet_pt_resolution'                                                
│                                   │     up_function: Function: jet_pt_resolution                                 
│                                   │     target: ('Jet', 'pt')                                                    
│                                   │     use: [('Jet', 'pt')]                                                     
│                                   │     symmetrise: True                                                         
│                                   │     op: 'mult'                                                               
│                                   │     type: 'object'                                                           
│                                   │   }                                                                          
│                                   │   [1]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'jet_pt_scale'                                                     
│                                   │     up_function: Function: jet_pt_scale                                      
│                                   │     target: ('Jet', 'pt')                                                    
│                                   │     symmetrise: True                                                         
│                                   │     op: 'mult'                                                               
│                                   │     type: 'object'                                                           
│                                   │   }                                                                          
│                                   │ ]                                                                            
└───────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────

                                               Configuration: STATISTICS                                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ cabinetry_config                  │ 'example/outputs/cabinetry/cabinetry_config.yaml'                            
└───────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────

                                                Configuration: PLOTTING                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                              ┃ Value                                                                         
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ process_colors                   │ {                                                                             
│                                  │   ttbar_semilep: '#907AD6'                                                    
│                                  │   signal: '#DABFFF'                                                           
│                                  │   ttbar_lep: '#7FDEFF'                                                        
│                                  │   ttbar_had: '#2C2A4A'                                                        
│                                  │   wjets: '#72A1E5'                                                            
│                                  │   ttbar: '#907AD6'                                                            
│                                  │ }                                                                             
│ process_labels                   │ {                                                                             
│                                  │   ttbar_semilep: '$t\bar{t}\,\textrm{(lepton+jets)}$'                         
│                                  │   signal: '$Z^{\prime} \rightarrow t\bar{t}$'                                 
│                                  │   ttbar_lep: '$t\bar{t}\,\textrm{(leptonic)}$'                                
│                                  │   ttbar_had: '$t\bar{t}\,\textrm{(hadronic)}$'                                
│                                  │   wjets: '$W+\textrm{jets}$'                                                  
│                                  │   ttbar: '$t\bar{t}$'                                                         
│                                  │ }                                                                             
│ process_order                    │ [                                                                             
│                                  │   [0]: 'ttbar'                                                                
│                                  │   [1]: 'ttbar_had'                                                            
│                                  │   [2]: 'ttbar_lep'                                                            
│                                  │   [3]: 'ttbar_semilep'                                                        
│                                  │   [4]: 'wjets'                                                                
│                                  │   [5]: 'signal'                                                               
│                                  │ ]                                                                             
└──────────────────────────────────┴───────────────────────────────────────────────────────────────────────────────

                                                Configuration: DATASETS                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ datasets                          │ [                                                                            
│                                   │   [0]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'signal'                                                           
│                                   │     directory: 'example/datasets/signal/m2000_w20/'                          
│                                   │     cross_section: 1.0                                                       
│                                   │     file_pattern: '*.txt'                                                    
│                                   │     tree_name: 'Events'                                                      
│                                   │     weight_branch: 'genWeight'                                               
│                                   │   }                                                                          
│                                   │   [1]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'ttbar_semilep'                                                    
│                                   │     directory: 'example/datasets/ttbar_semilep/'                             
│                                   │     cross_section: 364.31088                                                 
│                                   │     file_pattern: '*.txt'                                                    
│                                   │     tree_name: 'Events'                                                      
│                                   │     weight_branch: 'genWeight'                                               
│                                   │   }                                                                          
│                                   │   [2]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'ttbar_had'                                                        
│                                   │     directory: 'example/datasets/ttbar_had/'                                 
│                                   │     cross_section: 380.11432                                                 
│                                   │     file_pattern: '*.txt'                                                    
│                                   │     tree_name: 'Events'                                                      
│                                   │     weight_branch: 'genWeight'                                               
│                                   │   }                                                                          
│                                   │   [3]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'ttbar_l

In [3]:
# Let's look at the datasets config
display_config_table({"datasets": config["datasets"]},
                    expand=True)

# Make a modification
config["datasets"]["max_files"] = 1 # Limit to 1 files per dataset

print("=== SHOW ONLY CHANGES ===")
display_config_table({"datasets": config["datasets"]},
                    expand=True,
                    compare_with={"datasets": original_config["datasets"]},
                    show_only_changes=True)

cli_args = []  # No CLI overrides for this demo
full_config = load_config_with_restricted_cli(config, cli_args)

print("✅ Processed CLI arguments and loaded full configuration")
validated_config = Config(**full_config)  # This is the key validation step!

                                                Configuration: DATASETS                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ datasets                          │ [                                                                            
│                                   │   [0]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'signal'                                                           
│                                   │     directory: 'example/datasets/signal/m2000_w20/'                          
│                                   │     cross_section: 1.0                                                       
│                                   │     file_pattern: '*.txt'                                                    
│                                   │     tree_name: 'Events'                                                      
│                                   │     weight_branch: 'genWeight'                                               
│                                   │   }                                                                          
│                                   │   [1]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'ttbar_semilep'                                                    
│                                   │     directory: 'example/datasets/ttbar_semilep/'                             
│                                   │     cross_section: 364.31088                                                 
│                                   │     file_pattern: '*.txt'                                                    
│                                   │     tree_name: 'Events'                                                      
│                                   │     weight_branch: 'genWeight'                                               
│                                   │   }                                                                          
│                                   │   [2]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'ttbar_had'                                                        
│                                   │     directory: 'example/datasets/ttbar_had/'                                 
│                                   │     cross_section: 380.11432                                                 
│                                   │     file_pattern: '*.txt'                                                    
│                                   │     tree_name: 'Events'                                                      
│                                   │     weight_branch: 'genWeight'                                               
│                                   │   }                                                                          
│                                   │   [3]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'ttbar_l

=== SHOW ONLY CHANGES ===


                                          Configuration: DATASETS (1 changes)                                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━
┃ Key                        ┃ Value                                                               ┃      Status   
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━
│ max_files                  │ 1                                                                   │      CHANGED  
└────────────────────────────┴─────────────────────────────────────────────────────────────────────┴───────────────

✅ Processed CLI arguments and loaded full configuration


# 3. Output Manager & Directory Structure

The framework uses a centralized `OutputDirectoryManager` (`utils/output_manager.py`) to organize all analysis outputs.

In [4]:
from utils.output_manager import OutputDirectoryManager

output_manager = OutputDirectoryManager(
    root_output_dir=validated_config.general.output_dir,
    cache_dir=validated_config.general.cache_dir,
    metadata_dir=validated_config.general.metadata_dir,
    skimmed_dir=validated_config.general.skimmed_dir
)

print("✅ Created OutputDirectoryManager with validated paths")


16:04:44 INFO     Output directory manager initialized with root:                              ]8;id=361464;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/output_manager.py\output_manager.py]8;;\:]8;id=625353;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/output_manager.py#73\73]8;;\
                  /Users/moaly/Work/iris-hep/integration-challenge/cms/example/outputs                             

✅ Created OutputDirectoryManager with validated paths


## 4. Metadata Extraction

Metadata extraction runs `coffea`'s preprocessor to build analysis workitems, a `coffea`-compatible fileset and a summary of the processed `NanoAOD`s. Its outputs stay in memory, but are also stored in `JSON` files to allow re-runing subsequent steps without pre-processing every time.

In [5]:
from utils.metadata_extractor import NanoAODMetadataGenerator
from utils.datasets import ConfigurableDatasetManager

# Step 1: Create dataset manager instance
dataset_manager = ConfigurableDatasetManager(validated_config.datasets)

# Step 2: Create metadata generator instance
validated_config.general.run_metadata_generation=True
metadata_generator = NanoAODMetadataGenerator(
    dataset_manager=dataset_manager,
    output_manager=output_manager
)
# In the back this uses coffea's preprocess function
metadata = metadata_generator.run()

         INFO     Initialized dataset manager with 6 datasets                                        ]8;id=982917;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/datasets.py\datasets.py]8;;\:]8;id=878036;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/datasets.py#38\38]8;;\

         INFO     Starting metadata generation workflow...                                ]8;id=141435;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=622211;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py#404\404]8;;\

         INFO     Building fileset for process: signal                                    ]8;id=512508;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=934882;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py#165\165]8;;\

         INFO     Building fileset for process: ttbar_semilep                             ]8;id=219680;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=978087;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py#165\165]8;;\

         INFO     Building fileset for process: ttbar_had                                 ]8;id=844192;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=123374;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py#165\165]8;;\

...

Output()

16:05:00 INFO     WorkItems metadata saved to                                             ]8;id=743010;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=422516;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py#705\705]8;;\
                  /Users/moaly/Work/iris-hep/integration-challenge/cms/example/outputs/me                          
                  tadata/workitems.json                                                                            

         INFO     NanoAODs summary generated.                                             ]8;id=303310;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=699947;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py#544\544]8;;\

         INFO     NanoAODs summary written to                                             ]8;id=463049;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=206881;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py#476\476]8;;\
                  /Users/moaly/Work/iris-hep/integration-challenge/cms/example/outputs/me                          
                  tadata/nanoaods.json                                                                             

         INFO     Metadata generation complete.                                           ]8;id=217961;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=804964;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/metadata_extractor.py#416\416]8;;\

# 5. Skimming & Event Selection

The skimming step processes the workitems from previous step, and applies event and branch selections with `dask` and a `coffea`-like processor (see [Alex's issue](https://github.com/scikit-hep/coffea/issues/1393)). Currently the skimming results are in-memory events, but also stored on disk as `ROOT` files to avoid having to run this multiple times. The events are also cached in `.pkl` files for faster re-reading. Eventually this step needs to be integrated with subsequent steps in a complete `coffea` or `coffea`-like processor.

In [6]:
from utils.skimming import process_workitems_with_skimming

# Extract workitems and fileset from metadata generator
fileset = metadata_generator.fileset
workitems = metadata_generator.workitems

print(f"📊 Processing {len(workitems)} workitems across {len(fileset)} datasets")

# Disable caching for demonstration
validated_config.general.read_from_cache = False
validated_config.general.run_skimming = True

# Skim data with dask according to the workitems
processed_datasets = process_workitems_with_skimming(
    workitems,
    validated_config,
    output_manager,
    fileset,
    metadata_generator.nanoaods_summary
)

# Display the structure of processed datasets
print(f"\n✅ Skimming complete! Processed datasets structure x:")
for dataset_name, events_list in processed_datasets.items():
    print(f"  📁 {dataset_name} processed")
    for i, (events, metadata) in enumerate(events_list):
        print(f"    └── File {i+1}: {len(events)} events, {len(events.fields)} branches")

print(f"\n💾 Skimmed data saved to: {output_manager.get_skimmed_dir()}")

📊 Processing 82 workitems across 6 datasets


         INFO     Starting workitem preprocessing with 82 workitems                                 ]8;id=422579;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=939025;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#705\705]8;;\

         INFO     Initialized workitem-based skimming manager                                       ]8;id=852188;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=723675;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#372\372]8;;\

         INFO     Running skimming                                                                  ]8;id=318210;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=831841;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#720\720]8;;\

         INFO     Processing 82 workitems                                                           ]8;id=917141;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=205209;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#403\403]8;;\

...

         INFO     Cached events for ttbar_semilep__nominal                                          ]8;id=4454;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=463830;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#831\831]8;;\

         INFO     Found existing skimmed files for signal__nominal: 5                               ]8;id=371498;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=608291;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#524\524]8;;\

         INFO     Merged 5 files → 97109 events for signal__nominal                                 ]8;id=205394;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=911823;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#822\822]8;;\

         INFO     Cached events for signal__nominal                                                 ]8;id=768262;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=538937;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#831\831]8;;\


✅ Skimming complete! Processed datasets structure x:
  📁 data processed
    └── File 1: 94251 events, 8 branches
  📁 wjets__nominal processed
    └── File 1: 7313 events, 10 branches
  📁 ttbar_lep__nominal processed
    └── File 1: 82406 events, 10 branches
  📁 ttbar_had__nominal processed
    └── File 1: 1958 events, 10 branches
  📁 ttbar_semilep__nominal processed
    └── File 1: 96683 events, 10 branches
  📁 signal__nominal processed
    └── File 1: 97109 events, 10 branches

💾 Skimmed data saved to: /Users/moaly/Work/iris-hep/integration-challenge/cms/example/outputs/skimmed


In [7]:
# Enable caching to show how it speeds up subsequent runs
validated_config.general.read_from_cache = True
validated_config.general.run_skimming = False
# Run the skimming again - it will look for cached files
# if they don't exist, it will fallback to skim regularly
cached_datasets = process_workitems_with_skimming(
    workitems,
    validated_config,
    output_manager,
    fileset,
    metadata_generator.nanoaods_summary
)

# Show cache directory contents
import os
cache_files = os.listdir(output_manager.get_skimmed_dir())
print(f"💾 Cached files in skimmed directory: {len(cache_files)} files")
print(f"📁 Cache location: {output_manager.get_cache_dir()}")

         INFO     Starting workitem preprocessing with 82 workitems                                 ]8;id=391456;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=640616;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#705\705]8;;\

         INFO     Initialized workitem-based skimming manager                                       ]8;id=743514;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=920036;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#372\372]8;;\

         INFO     Reading from saved files                                                          ]8;id=983839;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=428442;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#725\725]8;;\

         INFO     Found existing skimmed files for data: 29                                         ]8;id=452711;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=883215;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#524\524]8;;\

         INFO     Loading cached events for data                                                    ]8;id=559422;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=467773;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#784\784]8;;\

...

         INFO     Loaded 96683 cached events                                                        ]8;id=721340;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=264627;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#788\788]8;;\

         INFO     Found existing skimmed files for signal__nominal: 5                               ]8;id=738865;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=195898;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#524\524]8;;\

         INFO     Loading cached events for signal__nominal                                         ]8;id=538159;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=539335;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#784\784]8;;\

         INFO     Loaded 97109 cached events                                                        ]8;id=593812;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=323823;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/skimming.py#788\788]8;;\

💾 Cached files in skimmed directory: 6 files
📁 Cache location: /private/tmp/integration


# 6. Analysis & Histogramming

This step encapsulates a few underlying function calls:
1. Global event selection is applied
2. If MVA training is configured and enabled, an MVA training will be triggered
3. Apply corrections from `correctionlib`
4. Ghost observables will be computed and added to event record
6. Channels will be built with their corresponding selections
7. Compute all observables (once nomina, once per systematic variation)
8. Create histograms

In [8]:
# Import the analysis class for the analysis
from analysis.nondiff import NonDiffAnalysis
from utils.output_files import save_histograms_to_root

validated_config.general.run_histogramming=True

# Initialize the analysis object
# This analysis object will handle histogram creation, observable computation, and systematic variations
nondiff_analysis = NonDiffAnalysis(validated_config, processed_datasets, output_manager)

print(f"🔬 Analysis initialized for {len(nondiff_analysis.processed_datasets)} datasets")

# Loop over each dataset and its associated event data
for dataset_name, events_list in nondiff_analysis.processed_datasets.items():
    print(f"📊 Processing dataset: {dataset_name}")
    # Each dataset contains a list of (events, metadata) tuples (in case of chunking)
    # Currently merging of chunks is default --> single tuple per dataset
    for events, metadata in events_list:
        print(f"   • Processing {len(events)} events with metadata: {metadata['process']}")
        # Process the events through the analysis pipeline:
        nondiff_analysis.process(events, metadata)
        
save_histograms_to_root(
    nondiff_analysis.nD_hists_per_region,
    output_file=nondiff_analysis.output_manager.get_histograms_dir() / "histograms.root",
)

print(f"📈 Generated histograms for channels: {[ch.name for ch in validated_config.channels]}")

🔬 Analysis initialized for 6 datasets
📊 Processing dataset: data
   • Processing 94251 events with metadata: data


         INFO     Applying selection for CMS_WORKSHOP in data with variation nominal                 ]8;id=338220;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=661014;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py#154\154]8;;\

         INFO     Number of weighted events in CMS_WORKSHOP: 20.00                                   ]8;id=602680;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=589697;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py#205\205]8;;\

         INFO     Number of raw events in CMS_WORKSHOP: 20                                           ]8;id=949868;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=300678;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py#208\208]8;;\

         INFO     Computing observable workshop_mtt                                                  ]8;id=593451;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=461267;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py#213\213]8;;\

...

         INFO     Number of weighted events in CMS_WORKSHOP: 15778.38                                ]8;id=215816;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=946243;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py#205\205]8;;\

         INFO     Number of raw events in CMS_WORKSHOP: 16069                                        ]8;id=270136;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=923241;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py#208\208]8;;\

         INFO     Computing observable workshop_mtt                                                  ]8;id=199624;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=17305;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/analysis/nondiff.py#213\213]8;;\

         INFO     Histograms successfully written to ROOT file                                  ]8;id=693653;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/output_files.py\output_files.py]8;;\:]8;id=59441;file:///Users/moaly/Work/iris-hep/integration-challenge/cms/utils/output_files.py#147\147]8;;\
                  /Users/moaly/Work/iris-hep/integration-challenge/cms/example/outputs/histogra                    
                  ms/histograms.root                                                                               

📈 Generated histograms for channels: ['CMS_WORKSHOP']


# 7. Statistical Analysis

The statistical analysis step performs a fit from the histograms and visualize results. This uses `cabinetry` with a manually created config. It also creates and stores the corresponding `pyhf` workspace.

**Note: the following fit is performed with the full CMS open-data dataset, not the small example here** 

In [9]:
import cabinetry
validated_config.statistics.cabinetry_config = "./example-old/outputs/cabinetry.
cabinetry_config = cabinetry.configuration.load(
    validated_config.statistics.cabinetry_config
)
data, fit_results, pre_fit_predictions, postfit_predictions = (
    nondiff_analysis.run_fit(cabinetry_config=cabinetry_config)
)
cabinetry.visualize.data_mc(
    pre_fit_predictions,
    data,
    close_figure=False,
    config=cabinetry_config,
    figure_folder=nondiff_analysis.output_manager.get_statistics_dir(),

)
cabinetry.visualize.data_mc(
    postfit_predictions,
    data,
    close_figure=False,
    config=cabinetry_config,
    figure_folder=nondiff_analysis.output_manager.get_statistics_dir(),
)
cabinetry.visualize.pulls(fit_results, close_figure=False, figure_folder=nondiff_analysis.output_manager.get_statistics_dir(),)

SyntaxError: unterminated string literal (detected at line 2) (1957373689.py, line 2)

## 8. Full Workflow

The above workflow is broken down for demonstration, but the simplest way to use it is by putting the main pieces together in one simple steering script and running it from the command-line with CLI overrides of config options. The current implementation is steered by `analysis.py` and can be simply executed with:

```sh
python analysis.py
```

---